# Stream an NWB File with fsspec
As you might have realized, NWB files are large. They take a lot of time to download and a lot of space on your drive. A convenient tool to mitigate this is **fsspec**. Fsspec allows you to *stream* the information from a file remotely without having to download it. This can be more efficient if you are only wanting to quickly examine a file or just need access to a portion of the file's contents. For more exensive analysis, it is still recommended that you download the file.

### Environment Setup

In [ ]:
from dandi import dandiapi
import dandi
import fsspec
import pynwb
import h5py
from fsspec.implementations.cached import CachingFileSystem
from nwbwidgets import nwb2widget

### Streaming Configuration
Here you can configure the stream. Browse the DANDI Archive for a Dandiset you're interested in and use its ID in `dandiset_id`. Set `filepath` to the path of the file you want to download within the dandiset. You can get this by navigating to the file you want to download on the DANDI Archive website and pressing on the `i` icon. There, you can copy the filepath from the field labeled `path`. Don't include a leading `/`.


If you're accessing an embargoed dandiset, you should set `authenticate` to True, and set `dandi_api_key` to your DANDI API Key, which can be found if you click on your profile icon in the top-right corner on the DANDI Archive website.

In [ ]:
dandiset_id = "000021"
filepath = "sub-699733573/sub-699733573_ses-715093703.nwb"
authenticate = False
dandi_api_key = ""

In [ ]:
if authenticate:
    client = dandiapi.DandiAPIClient(token=dandi_api_key)
else:
    client = dandiapi.DandiAPIClient()
my_dandiset = client.get_dandiset(dandiset_id)

print(f"Got dandiset {my_dandiset}")

In [ ]:
file = my_dandiset.get_asset_by_path(filepath)
base_url = file.client.session.head(file.base_download_url)
file_url = base_url.headers['Location']

print(f"Retrieved file url {file_url}")

### Stream Your File
First, this creates a virtual filesystem based on the http protocol and specifies using caching to save accessed data to RAM. Then it opens the file remotely through the virtual filesystem.

In [ ]:
fs = CachingFileSystem(
    fs=fsspec.filesystem("http"),
    cache_storage="nwb-cache",  # Local folder for the cache
)

f = fs.open(file_url, "rb")
file = h5py.File(f)
io = pynwb.NWBHDF5IO(file=file, mode='r', load_namespaces=True)
nwb = io.read()

### Interacting with Remote File
Once the file has been opened remotely, you can explore the file as you wish via `print` statements, or you can view the whole thing with `NWBWidgets` like we showed in *Exploring an NWB File*.

In [ ]:
### uncomment these to view aspects of the file
### note that not all these properties exist for all nwb files
# nwb.identifier
# nwb.processing
# nwb.acquisition["events"]
# nwb.intervals["trials"]
# nwb.stimulus["StimulusPresentation"]
# nwb.electrodes

In [ ]:
nwb2widget(nwb)